# Log traces


In [1]:
from opik.integrations.openai import track_openai
from openai import OpenAI

client = OpenAI()
client = track_openai(client)

### Using an integration

In [2]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role":"user", "content": "Hello, world!"}
    ]
)

OPIK: Started logging traces to the "Default Project" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=01983a90-acbb-7a11-9a6f-fe39ea3e2495&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


### Using function decorators

In [4]:
import opik
import openai

@opik.track
def retrieve_context(input_text):
    # Your retrieval logic here, here we are just returning a hardcoded list of strings
    context =[
        "What specific information are you looking for?",
        "How can I assist you with your interests today?",
        "Are there any topics you'd like to explore or learn more about?",
    ]
    return context
@opik.track
def generate_response(input_text, context):
    full_prompt = (
        f" If the user asks a question that is not specific, use the context to provide a relevant response.\n"
        f"Context: {', '.join(context)}\n"
        f"User: {input_text}\n"
        f"AI:"
    )
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": full_prompt}]
    )
    return response.choices[0].message.content

@opik.track(name="my_llm_application")
def llm_chain(input_text):
    context = retrieve_context(input_text)
    response = generate_response(input_text, context)
    return response
# Use the LLM chain
result = llm_chain("Hello, how are you?")
print(result)

Hello! I'm here to assist you with any specific information or topics you'd like to explore. How can I help you today?


The `@track` decorator will only track the input and output of the decorated function. If you are using OpenAI, we recommend you also use the `track_openai` function to track the LLM call as well as token usage:

```python
from opik.integrations.openai import track_openai
from openai import OpenAI

client = OpenAI()
client = track_openai(client)

```

### Scoring traces

Bạn có thể log (ghi lại) các điểm phản hồi cho các **trace** bằng cách sử dụng hàm `opik_context.update_current_trace`. Điều này có thể hữu ích nếu có một số **metrics** (chỉ số) đã được báo cáo như một phần của `chain` hoặc `agent` của bạn

Để dễ hiểu, hãy coi mỗi yêu cầu bạn gửi đến ứng dụng LLM của mình là một "nhiệm vụ". **`Trace`** chính là **toàn bộ hành trình xử lý** nhiệm vụ đó, từ đầu đến cuối.

-----

### 1\. "Trace" là gì trong ngữ cảnh LLM?

Một `trace` ghi lại tất cả các bước mà ứng dụng của bạn đã thực hiện để tạo ra một câu trả lời cuối cùng. Trong một ứng dụng LLM phức tạp, một `trace` không chỉ là một lần gọi API duy nhất. Nó có thể bao gồm nhiều bước, ví dụ:

  * **Đầu vào của người dùng:** "Tóm tắt bài báo này cho tôi."
  * **Bước 1: Gọi Tool/API:** Hệ thống gọi một công cụ để lấy nội dung từ URL của bài báo.
  * **Bước 2: Xử lý dữ liệu:** Trích xuất văn bản chính từ trang web.
  * **Bước 3: Gọi LLM:** Gửi văn bản đã trích xuất đến một LLM (ví dụ: GPT-4, Gemini) với `prompt` "Hãy tóm tắt văn bản sau: ...".
  * **Bước 4: Phân tích kết quả (tùy chọn):** Hệ thống có thể chạy một phân tích cảm xúc (sentiment analysis) trên bản tóm tắt.
  * **Đầu ra cuối cùng:** Trả về bản tóm tắt cho người dùng.

Toàn bộ chuỗi sự kiện này được gói gọn trong một **`trace`**. Việc theo dõi (tracking) các `trace` giúp bạn có một cái nhìn chi tiết và minh bạch về cách ứng dụng của mình hoạt động "dưới mui xe".

-----

### 2\. "Scoring" (Chấm điểm) một Trace nghĩa là gì?

**Scoring** là quá trình bạn **gán một hoặc nhiều điểm số (score) hoặc nhãn (label) cho một `trace`** để đánh giá chất lượng hoặc hiệu suất của nó. Đây không phải là điểm số do LLM tự động tạo ra, mà là các **`metrics` (chỉ số)** mà *bạn* định nghĩa và ghi lại.

Các loại điểm số bạn có thể `log` (ghi lại) bao gồm:

  * **Điểm phản hồi của người dùng:** Người dùng có bấm nút "thích" 👍 hoặc "không thích" 👎 không?
  * **Độ chính xác (Accuracy):** Câu trả lời có đúng sự thật không? (Ví dụ: bạn có thể chấm điểm 1 nếu đúng, 0 nếu sai).
  * **Mức độ liên quan (Relevance):** Kết quả tìm kiếm hoặc câu trả lời có liên quan đến câu hỏi không?
  * **Độc hại (Toxicity):** Câu trả lời có chứa nội dung không phù hợp không?
  * **Chỉ số nghiệp vụ (Business Metrics):** Kết quả có giúp người dùng hoàn thành một tác vụ cụ thể không (ví dụ: đặt hàng thành công)?

-----

### 3\. Vai trò của `opik_context.update_current_trace`

Đây chính là công cụ kỹ thuật để bạn thực hiện việc "scoring".

Khi một `chain` hoặc `agent` của bạn đang chạy, nó đang ở trong một `trace` đang hoạt động. Hàm `opik_context.update_current_trace` cho phép bạn **thêm thông tin vào `trace` đó ngay tại thời điểm nó đang chạy**.

Cụ thể, bạn có thể thêm các `score` mà bạn đã tính toán được. Ví dụ, sau khi LLM trả về kết quả, bạn có thể chạy một hàm riêng để kiểm tra xem câu trả lời có chứa thông tin trích dẫn nguồn hay không, và sau đó gọi `update_current_trace` để ghi lại điểm "có trích dẫn" = 1 hoặc 0.

-----

### 4\. Ví dụ thực tế trong một `chain`

Hãy tưởng tượng bạn có một `chain` dịch thuật và đánh giá cảm xúc:

1.  **Input:** "I love this new product, it's amazing\!"
2.  **Chain Step 1 (Dịch thuật):** Dịch sang tiếng Việt -\> "Tôi yêu sản phẩm mới này, nó thật tuyệt vời\!"
3.  **Chain Step 2 (Đánh giá cảm xúc):** Phân tích câu tiếng Việt và cho ra kết quả `sentiment: "Tích cực"`.
4.  **Chain Step 3 (Tự đánh giá):** Bạn có một hàm nội bộ để kiểm tra xem kết quả dịch có giữ nguyên ý nghĩa không. Hàm này trả về `{ "translation_quality": 0.95 }`.

Lúc này, bạn có thể dùng `opik_context.update_current_trace` để ghi lại điểm số này:

```python
# Giả sử đây là đoạn code trong chain của bạn
from opik import opik_context

# ... code dịch thuật và đánh giá cảm xúc ...

# Tự tính toán một metric
translation_score = calculate_translation_quality(original_text, translated_text) # Giả sử trả về 0.95

# Ghi lại điểm số này vào trace hiện tại
opik_context.update_current_trace({
    "scores": {
        "translation_quality": translation_score,
        "sentiment_score": 1 # 1 cho Tích cực, -1 cho Tiêu cực
    }
})

```

Khi bạn xem lại `trace` này trên giao diện của `opik`, bạn sẽ thấy không chỉ các bước thực thi mà còn cả các điểm số `translation_quality` và `sentiment_score` mà bạn đã ghi lại.

### Lợi ích chính

  * **Gỡ lỗi (Debugging):** Dễ dàng xác định bước nào trong `chain` hoặc `agent` đang hoạt động kém hiệu quả.
  * **Đánh giá chất lượng:** Thu thập dữ liệu có cấu trúc về hiệu suất của mô hình theo các tiêu chí quan trọng đối với bạn.
  * **Cải tiến liên tục:** Dựa vào các `score` đã thu thập, bạn có thể tinh chỉnh `prompt`, thay đổi mô hình, hoặc điều chỉnh logic của `agent` để cải thiện chất lượng đầu ra theo thời gian.

### Logging additional data


Như đã đề cập ở trên, `decorator` **`@track`** chỉ ghi nhật ký **đầu vào (`input`)** và **đầu ra (`output`)** của hàm được trang trí. Nếu bạn muốn ghi nhật ký dữ liệu bổ sung, bạn có thể sử dụng hàm **`update_current_span`** và hàm **`update_current_trace`** để cập nhật thủ công `span` và `trace`:

```python
from opik import track, opik_context

@track
def llm_chain(input_text):
    # LLM chain code
    # ...

    # Update the trace
    opik_context.update_current_trace(
        tags=["llm_chatbot"],
    )

    # Update the span
    opik_context.update_current_span(
        name="llm_chain"
    )

```

### Configuring the project name

```python
import opik

@opik.track(project_name="my_project")
def my_function(input):
    # Function code
    return input

```

or 

```python
import os

os.environ["OPIK_PROJECT_NAME"] = "my_project"

```

### Flushing the trace

"Flushing the trace" (Xả trace) có nghĩa là **buộc tất cả dữ liệu theo dõi (trace data) đã được thu thập phải được gửi đi ngay lập tức** thay vì chờ đợi để gửi theo từng đợt.

Thông thường, để tối ưu hiệu suất, các hệ thống giám sát sẽ thu thập dữ liệu vào một bộ đệm (buffer) và gửi chúng đi một cách bất đồng bộ trong nền. Tuy nhiên, việc "xả trace" sẽ đảm bảo dữ liệu của bạn không bị mất, đặc biệt là trong các ứng dụng chạy nhanh và kết thúc đột ngột.

-----

### \#\# Hoạt động Mặc định (`flush=False`) ⚙️

Để không làm chậm ứng dụng chính của bạn, các công cụ theo dõi như `opik` thường hoạt động theo cơ chế bất đồng bộ:

1.  **Thu thập dữ liệu:** Khi `chain` hoặc `agent` của bạn chạy, dữ liệu về các bước thực thi, thời gian, và metadata được ghi vào một bộ đệm tạm thời trong bộ nhớ.
2.  **Gửi theo lô (Batching):** Một tiến trình chạy nền (background process) sẽ định kỳ gom dữ liệu từ bộ đệm và gửi chúng thành từng lô đến máy chủ giám sát.

Cách này rất hiệu quả vì nó giảm thiểu số lượng yêu cầu mạng và không bắt ứng dụng của bạn phải chờ đợi.

-----

### \#\# Khi nào cần "Xả Trace" (`flush=True`) ⚡

Vấn đề với cơ chế mặc định là nếu ứng dụng của bạn kết thúc quá nhanh (ví dụ: một kịch bản dòng lệnh, một hàm serverless), chương trình chính có thể thoát **trước khi** tiến trình nền kịp gửi dữ liệu đi. Kết quả là bạn sẽ bị mất `trace` đó.

Việc thiết lập **`flush=True`** trong decorator `@track` giải quyết vấn đề này.

```python
from opik import track

@track(flush=True)
def my_short_running_task():
    # ... các bước xử lý nhanh ...
    return "Done"

# Khi hàm này chạy xong, chương trình sẽ đợi cho đến khi 
# toàn bộ dữ liệu của trace này được gửi đi thành công.
my_short_running_task()
```

Khi `flush=True`, sau khi hàm được `decorate` thực thi xong, chương trình sẽ **dừng lại và chờ** cho đến khi tất cả dữ liệu của `trace` đó được gửi đi và nhận được xác nhận từ máy chủ.

**Các trường hợp sử dụng chính:**

  * **Hàm Serverless:** Như AWS Lambda hay Google Cloud Functions, vì môi trường thực thi có thể bị đóng băng hoặc tắt ngay sau khi hàm hoàn thành.
  * **Kịch bản ngắn (Scripts):** Các file Python chạy một lần để xử lý dữ liệu rồi thoát.
  * **Gỡ lỗi (Debugging):** Khi bạn muốn thấy `trace` xuất hiện trên dashboard ngay lập tức để kiểm tra.

-----

### \#\# Đánh đổi cần cân nhắc

  * **`flush=False` (Mặc định):**

      * ✅ **Ưu điểm:** Hiệu suất cao, độ trễ (latency) của ứng dụng thấp.
      * ⚠️ **Nhược điểm:** Có nguy cơ mất dữ liệu nếu ứng dụng kết thúc đột ngột.

  * **`flush=True`:**

      * ✅ **Ưu điểm:** Đảm bảo 100% dữ liệu được ghi lại.
      * ⚠️ **Nhược điểm:** Tăng nhẹ độ trễ cho tác vụ vì phải chờ mạng, có thể ảnh hưởng đến các ứng dụng đòi hỏi phản hồi tức thì.

### Disabling automatic logging of function input and output

```python
import opik

@opik.track(capture_input=False, capture_output=False)
def llm_chain(input_text):
    # LLM chain code
    return input_text

```

### Disable all tracing

```python
import os

os.environ["OPIK_TRACK_DISABLE"] = "true"

```

## Using the low-level Opik client


In [6]:
from opik import Opik
client = Opik(project_name="Opik client demo")

In [7]:
# Create a trace
trace = client.trace(
    name="my_trace",
    input={"user_question": "Hello, how are you?"},
    output={"response": "Comment ça va?"}
)
# Add a span
trace.span(
    name="Add prompt template",
    input={"text": "Hello, how are you?", "prompt_template": "Translate the following text to French: {text}"},
    output={"text": "Translate the following text to French: hello, how are you?"}
)
# Add an LLM call
trace.span(
    name="llm_call",
    type="llm",
    input={"prompt": "Translate the following text to French: hello, how are you?"},
    output={"response": "Comment ça va?"}
)
# End the trace
trace.end()

OPIK: Started logging traces to the "Opik client demo" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=01983ab0-44e0-7852-adae-5805500e69ed&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.
